In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rc,  font_manager
import matplotlib
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import statistics
matplotlib.rcParams['axes.unicode_minus'] = False
fontname = "C:/Windows/Fonts/malgun.ttf"
font_name = font_manager.FontProperties(fname=fontname).get_name()
rc('font', family=font_name)

In [9]:
get_data = pd.read_csv('../data/TEMP/경락가격/애호박/애호박 월별 데이터와 평균편차(전국포함).csv', index_col=None)

In [10]:
get_data.head()

,Date,서울 총거래물량,서울 총거래금액,서울 평균단가,서울 평균단가 편차,부산 총거래물량,부산 총거래금액,부산 평균단가,부산 평균단가 편차,대구 총거래물량,...,대전 총거래물량,대전 총거래금액,대전 평균단가,대전 평균단가 편차,광주 총거래물량,광주 총거래금액,광주 평균단가,광주 평균단가 편차,전국 평균단가,전국 평균단가 편차
0,2020-01-01,2780377.65,9830108400,3535.529931,22452.542192,314022.5,1158346710,3688.737941,25041.342430,810553.0,...,430426.0,1337955110,3108.443983,20480.408708,478129.0,1644281360,3438.991067,22661.976441,3324.666248,-1372.673763
1,2020-02-01,2720669.88,9024401700,3316.977839,22671.094283,331341.0,1143201466,3450.226401,25279.853969,827334.0,...,406261.0,1228049490,3022.809204,20729.659259,451781.0,1441064160,3189.740516,22911.226992,3127.164424,-1175.171940
2,2020-03-01,3739541.28,9722242800,2599.849038,23388.223084,435282.4,1220358529,2803.601820,25926.478550,978456.0,...,520843.0,1246237300,2392.731207,21349.931079,587348.0,1509172300,2569.468697,23531.498812,2496.717006,-544.724521
3,2020-04-01,5703989.95,7969557560,1397.189972,24590.882150,607195.5,995696080,1639.827831,27090.252539,1427859.0,...,615618.0,884077957,1436.082046,22478.016491,800667.0,1154068030,1441.383284,24659.584224,1386.114372,565.878113
4,2020-05-01,5487463.43,7107073600,1295.147328,24692.924794,644394.2,875168986,1358.126727,27371.953643,1371680.0,...,631163.0,818889984,1297.430274,22684.541148,759615.0,938017136,1234.858627,24866.108881,1267.346727,684.645757


In [11]:
get_data['전국 총거래물량'] = get_data['서울 총거래물량'] + get_data['부산 총거래물량'] + get_data['대구 총거래물량'] + get_data['대전 총거래물량'] + get_data['광주 총거래물량']
get_data['전국 총거래금액'] = get_data['서울 총거래금액'] + get_data['부산 총거래금액'] + get_data['대구 총거래금액'] + get_data['대전 총거래금액'] + get_data['광주 총거래금액']

In [12]:
seoul_columns = ['Date'] + list(get_data.columns[get_data.columns.str.contains('서울')])
seoul_data = get_data[seoul_columns]

busan_columns = ['Date'] + list(get_data.columns[get_data.columns.str.contains('부산')])
busan_data = get_data[busan_columns]

daegu_columns = ['Date'] + list(get_data.columns[get_data.columns.str.contains('대구')])
daegu_data = get_data[daegu_columns]

daejeon_columns = ['Date'] + list(get_data.columns[get_data.columns.str.contains('대전')])
daejeon_data = get_data[daejeon_columns]

gwangju_columns = ['Date'] + list(get_data.columns[get_data.columns.str.contains('광주')])
gwangju_data = get_data[gwangju_columns]

aways_columns = ['Date'] + list(get_data.columns[get_data.columns.str.contains('전국')])
aways_data = get_data[aways_columns]

In [13]:
aways_data = aways_data[['Date', '전국 총거래물량', '전국 총거래금액', '전국 평균단가', '전국 평균단가 편차']]

In [14]:
seoul_data = seoul_data.rename(columns={'서울 총거래물량':'총거래물량', '서울 총거래금액':'총거래금액', '서울 평균단가':'평균단가', '서울 평균단가 편차':'평균단가 편차'})
busan_data = busan_data.rename(columns={'부산 총거래물량':'총거래물량', '부산 총거래금액':'총거래금액', '부산 평균단가':'평균단가', '부산 평균단가 편차':'평균단가 편차'})
daegu_data = daegu_data.rename(columns={'대구 총거래물량':'총거래물량', '대구 총거래금액':'총거래금액', '대구 평균단가':'평균단가', '대구 평균단가 편차':'평균단가 편차'})
daejeon_data = daejeon_data.rename(columns={'대전 총거래물량':'총거래물량', '대전 총거래금액':'총거래금액', '대전 평균단가':'평균단가', '대전 평균단가 편차':'평균단가 편차'})
gwangju_data = gwangju_data.rename(columns={'광주 총거래물량':'총거래물량', '광주 총거래금액':'총거래금액', '광주 평균단가':'평균단가', '광주 평균단가 편차':'평균단가 편차'})
aways_data = aways_data.rename(columns={'전국 총거래물량':'총거래물량', '전국 총거래금액':'총거래금액', '전국 평균단가':'평균단가', '전국 평균단가 편차':'평균단가 편차'})

In [17]:
seoul_data['지역'] = '서울'
busan_data['지역'] = '부산'
daegu_data['지역'] = '대구'
daejeon_data['지역'] = '대전'
gwangju_data['지역'] = '광주'
aways_data['지역'] = '전국'

In [19]:
all_data = pd.concat([seoul_data, busan_data, daegu_data, daejeon_data, gwangju_data, aways_data])

In [21]:
all_data.to_csv('../data/TEMP/경락가격/애호박/애호박 컬럼 통합한 데이터와 표준편차.csv', index=False, encoding='UTF-8')